<a href="https://colab.research.google.com/github/Eun-Tack/kaggle_data/blob/main/topic8_card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Kaggle 데이터 활용 데이터 분석 실습.**
주제 8. 카드 부정사용 탐지 데이터.
kaggle 데이터셋 주소 : https://www.kaggle.com/kartik2112/fraud-detection



> 카드 부정 사용 탐지 데이터는 수집 가능한 변수를 기반으로 분석자의 여러 가지 부정 사용을 탐지할 수 있는 변수에 대하여 생각하고 깆존의 데이터를 기반으로 새로운 데이터를 만들고 이를 분석에 활용하는 연습을 중점으로 하였습니다.

기본 변수에 대한 설명은 위 Kaggle 데이터셋 주소를 참고하여 주시기 바랍니다.

또한 런타임 유형은 TPU로 진행하여주시기 바랍니다.
딥러닝 분석에 있어 많은 시간이 소요.

In [ ]:
# CSV 파일 열기 위해 실행
import pandas as pd
raw_card_data=pd.read_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/데이터(원데이터, spss 데이터)/8. 카드부정사용 데이터.csv')
print(raw_card_data) # 데이터셋 확인
print(raw_card_data.dtypes) # 데이터셋 데이터 타입 확인
print(raw_card_data.shape) # 데이터셋 행과 열 확인

In [ ]:
'''
필요한 열만 가져다가 필요한 데이터를 만들면서 접근하는 방식으로 진행
Unnamed: 0                 int64 -> V1으로 이름 변경: 데이터 병합에 있어 Key값으로 사용
trans_date_trans_time     object -> 데이터 타입으로 변경 필요
cc_num                     int64 -> 고객 거래 구분을 위해 사용
merchant                  object -> 범주가 너무 많기 때문에 실습에 부적합(학습 시간 증대)하기에 삭제
category                  object 
amt                      float64 -> 평균 금액으로 사용
first                     object -> 분석에 불필요 삭제
last                      object -> 분석에 불필요 삭제
gender                    object 
street                    object -> 범주 많기 때문에 삭제
city                      object -> 범주 많기 때문에 사용은 X 하지만 city_pop을 이용하여 State_pop을 구하기 위해 key값으로 활용
state                     object
zip                        int64 -> 우편 번호 불필요
lat                      float64 
long                     float64
city_pop                   int64 -> State_pop 계산에 사용
job                       object -> 너무 다양. 실제 실무에서는 사용할 수 있을 것으로 판단.
dob                       object -> 생년월일로 연령을 구해서 사용할 수 있을 것으로 판단 (연령 별 categiry 등). 하지만 이번 실습에서는 제외
trans_num                 object -> 삭제
unix_time                  int64 -> 삭제
merch_lat                float64
merch_long               float64
is_fraud                   int64 -> 종속변수
'''

In [ ]:
# 데이터 합치는 key 값은 데이터 순서를 의미하는 'unnamed: 0'로 사용하는데 열의 이름을 변경하여 사용하겠음.
# 본 실습 교재에서는 'V1'이었기 때문에 이름을 여기에 맞춰주겠음
raw_card_data.rename(columns={'Unnamed: 0':'V1'}, inplace=True)
raw_card_data

In [ ]:
# 생성된 파일은 별도로 저장
# 인덱스를 의히하였던 V1을 key값으로 계속해서 사용할 것이기 때문에 저장할 때 index=False로 설정, 만약 설정하지 않으면 계속해서 unamed: 0가 생성됨
raw_card_data.to_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/(FT_작성)R_python_Code/raw_card_data.csv', header=True, index=False,encoding='utf-8-sig')

In [ ]:
# 카드 사용자의 거주지와 카드 사용지의 거리를 계산
# 필요한 데이터 : 'V1', 'lat', 'long', 'merch_lat','merch_long'
distance_data=raw_card_data[['V1','lat','long','merch_lat','merch_long']]
distance_data

In [ ]:
# 위경도 리스트 만들기
cu_lat=[]
cu_long=[]
mer_lat=[]
mer_long=[]
for i in range(len(distance_data)):
    cu_lat.append(distance_data['lat'][i])
    cu_long.append(distance_data['long'][i])
    mer_lat.append(distance_data['merch_lat'][i])
    mer_long.append(distance_data['merch_long'][i])
print(cu_lat)

In [ ]:
# 위경도 거리 차이 구하는 함수 만들기
# 삼각함수 사용을 위한 패키지 탑재
from math import sin, cos, sqrt, atan2, radians

R=6373.0 #지구 반지금
def lat_long_distance(a1,b1,a2,b2):
    x1=radians(a1)
    y1=radians(b1)
    x2=radians(a2)
    y2=radians(b2)
    differ_long=y2-y1
    differ_lat=x2-x1
    a = sin(differ_lat / 2)**2 + cos(x1) * cos(x2) * sin(differ_long / 2)**2 
    c = 2 * atan2(sqrt(a), sqrt(1 - a))    
    return R * c

In [ ]:
#print(lat_long_distance(36.0788,-81.1781,36.01129,-82.0483)) # 교재에 있는 값으로 테스트한 결과, 78.682로 유사한 것으로 확인

differ_km=[]
for i,j,k,l in zip(cu_lat,cu_long,mer_lat,mer_long):
    differ_km.append(lat_long_distance(i,j,k,l))

In [ ]:
differ_km

In [ ]:
# 리스트 형태의 differ_km를 distance_data의 'differ_km' 변수로써 대입
distance_data['differ_km']=differ_km
distance_data

In [ ]:
# 데이터 저장

distance_data.to_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/(FT_작성)R_python_Code/distance_data.csv', header=True, index=False,encoding='utf-8-sig')

In [ ]:
# 고객별 최근 거래와의 시간과 결제지 거리 차이를 구하기 위해 필요한 데이터셋 로딩
# 필요한 데이터 V1, trans_date_trans_time, cc_num, merch_lat, merch_long
time_distance_data=raw_card_data[['V1', 'trans_date_trans_time', 'cc_num', 'merch_lat', 'merch_long']]
print(time_distance_data)

In [ ]:
# 고객별로 거래 시간에 맞춰 설정
# 이중으로 고객을 구분하기 위한 'cc_num'과 거래 순서를 구분하기 위한 'trans_date_trans_time'데이터를 올림차순으로 정렬
time_distance_sort=time_distance_data.sort_values(['cc_num', 'trans_date_trans_time'], ascending=[True, True])
time_distance_sort 
# 이때 index(가장 왼편의 숫자와 V1이 일치하는 것을 확인할 수 있는데 행렬의 위치정보는 index의 숫자를 따라가는데 행렬의 위치와 인덱스가 달라 반복문에 어려움.

In [ ]:
# 정렬된 그대로 데이터 저장

time_distance_sort.to_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/(FT_작성)R_python_Code/time_distance_sort.csv', header=True, index=False,encoding='utf-8-sig')

In [ ]:
# 정렬된 상태에서 인덱스 초기화를 위해서 저장 후 로딩
# V1 초기 인덱스와 인덱스가 다르게 부여된 것을 확인할 수 있음. 이 작업을 하지 않으면 반복문을 만들때 계산 오류 발생
time_distance_sort=pd.read_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/(FT_작성)R_python_Code/time_distance_sort.csv')
time_distance_sort # V1과 인덱스가 다른 것을 확인할 수 있음 V1은 데이터 병합의 key값이고 index는 행렬의 데이터 위치를 찾기위한 위치정보임

In [ ]:
# 소비자 별로 과거 거래 시간과의 차이를 구할 것. 바로 사용되면 부정 사용을 의심
differ_time_past=time_distance_sort[['V1','trans_date_trans_time','cc_num']]
from datetime import datetime # 시간인 object형이었기 때문에 이를 date로 인식하기 위한 패키지를 탑재
differ_time_past['trans_date_trans_time'] = pd.to_datetime(differ_time_past['trans_date_trans_time'])

# 최근 사용지와 비교했을 때 너무 멀면 의심할 수 있을 것으로 판단하여 최근 거래지와의 거리를 계산할 것
differ_distance_past=time_distance_sort[['V1','cc_num','merch_lat','merch_long']]
print(differ_time_past)
print(differ_distance_past)

In [ ]:
# 최근 거래 시간과의 차이를 계산하기 위한 반복문을 만들 것

differ_time_past['differ_time']=1 # 1로 설정함으로써 계산식이 제대로 작동했는지를 확인할 것. 만약 0으로 하면 수식 안의 0인지 초기값의 0인지 혼동이 있을 수 있음
# i=0 즉 첫 번째 행은 0 => 기존 거래가 없는 데이터이기 때문에 / i와 i-1(직전거래)의  고객 넘버(같은 사용자)라면 최근 거래 시간의 차이를 구하라. 아니라면 0 (고객이 다르면 초기값 0)
# 만약 index 초기화를 하지 않았다면 위치가 정렬한 행렬의 위치 정보와 다르기 때문에 계산이 어려움.
for i, row in differ_time_past.iterrows():
    if i==0:
        differ_time_past.at[i,'differ_time']=0
    elif differ_time_past.at[i,'cc_num']==differ_time_past.at[i-1,'cc_num']:
        differ_time_past.at[i,'differ_time']=pd.Timedelta(differ_time_past.at[i,'trans_date_trans_time']-differ_time_past.at[i-1,'trans_date_trans_time']).seconds/60
    else :
       differ_time_past.at[i,'differ_time']=0

In [ ]:
# 생성된 데이터 확인하기
differ_time_past

In [ ]:
# 생성된 데이터셋은 저장
differ_time_past.to_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/(FT_작성)R_python_Code/differ_time_past.csv', header=True, index=False,encoding='utf-8-sig')

In [ ]:
# 최근 거래지와의 거리를 계산.
# 앞서 살펴봤던 거주지와 거래지 거리 차이 계산식과 위에서 살펴봤던 반복문을 이용하여 변수 생성
from math import sin, cos, sqrt, atan2, radians

R=6373.0 #지구 반지금
def lat_long_distance(a1,b1,a2,b2):
    x1=radians(a1)
    y1=radians(b1)
    x2=radians(a2)
    y2=radians(b2)
    differ_long=y2-y1
    differ_lat=x2-x1
    a = sin(differ_lat / 2)**2 + cos(x1) * cos(x2) * sin(differ_long / 2)**2 
    c = 2 * atan2(sqrt(a), sqrt(1 - a))    
    return R * c

differ_distance_past['differ_past_km']=1
for i, row in differ_distance_past.iterrows():
    if i==0:
        differ_distance_past.at[i,'differ_past_km']=0
    elif differ_distance_past.at[i,'cc_num']==differ_distance_past.at[i-1,'cc_num']:
        differ_distance_past.at[i,'differ_past_km']=lat_long_distance(differ_distance_past.at[i-1,'merch_lat'],differ_distance_past.at[i-1,'merch_long'],differ_distance_past.at[i,'merch_lat'],differ_distance_past.at[i,'merch_long']) 
    else :
       differ_distance_past.at[i,'differ_past_km']=0

In [ ]:
# 생성된 데이터 확인
differ_distance_past

In [ ]:
# 생성된 데이터는 저장
differ_distance_past.to_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/(FT_작성)R_python_Code/differ_distance_past.csv', header=True, index=False,encoding='utf-8-sig')

In [ ]:
# 사용자의 평균 결제 금액과의 차액 비교
'''
고객 별로 amt의 평균을 구할 것.
고객 groupby mean 해서 데이터셋 만들기
그 다음에 key값(cc_num) 토대로 붙이기
그 다음 차이 구하면 될 듯
'''
# 필요한 변수 V1, cc_num, amt
cc_amt=raw_card_data[['V1','cc_num','amt']] # 최종적으로 저장할 데이터셋
cc_amt_col=raw_card_data[['cc_num','amt']] # groupby를 이용하는 과정에서 'V1'은 불필요하기 떄문에 제외한 순수 연산을 위한 데이터셋 만들기
cc_amt_group=cc_amt_col.set_index(['cc_num']).groupby(level=[0]) # cc_num을 기준으로 데이터셋 만들기
cc_amt_mean=cc_amt_group.mean() # key값인 cc_num을 기준으로 갑은 key에서의 변수들의 평균을 구하기 때문에 amt의 평균을 구하게 됨.
cc_amt_mean.rename(columns={'amt':'amt_mean'}, inplace=True) # cc_amt와 데이터셋 병합의 과정에서 이름 중복을 피하기 위해 변수명을 'amt_mean'으로 변경
cc_amt_mean # 데이터 확인
# 데이터 프레임 병합
cc_amt_merge=pd.merge(cc_amt,cc_amt_mean, how='left',on='cc_num') # cc_num을 key값으로 left조인하며 cc_amt의 데이터셋에 cc_amt_mat의 데이터가 입력되는 방식
# 정렬해서고객별로 잘 정리되었는지 확인
cc_amt_merge_sort=cc_amt_merge.sort_values(['cc_num'], ascending=[True]) # 같은 고객이 동일한 평균이 들어가있는지 확인
cc_amt_merge['differ_amt']=cc_amt_merge['amt_mean']-cc_amt_merge['amt'] # 평균 금액과 거래 금액의 차이를 구함. 평균 소비보다 거래 금액이 많이 크면 부정 사용일거라는 가설.

In [ ]:
cc_amt_merge # 데이터 확인
# 생성된 데이터셋 저장
cc_amt_merge.to_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/(FT_작성)R_python_Code/differ_amt.csv', header=True, index=False,encoding='utf-8-sig')

In [ ]:
# state_pop
'''
state group_by city pop sum
is_fraud/state_pop 계산해서 변수 만들어 두기
'''
#필요한 변수 V1, city_pop, state, is_fraud
state_pop=raw_card_data[['V1','state']] # 나중에 전체 데이터셋을 병합하기 위한 데이터셋
state_pop_col=raw_card_data[['state','city','city_pop']] # state_pop을 계산하기 위한 데이터셋
city_group=state_pop_col.set_index(['state','city']).groupby(level=[0,1]) # 이 중 인덱스를 실시하여 도시별 인구수 평균 계산을 통해 중복된 city_pop을 제거.
city_group_mean=city_group.mean() # 중복된 city_pop을 제외하여 하나의 city에 따른 하나의 city_pop이 만들어 질 수 있도록
# 인덱스 초기화를 위해 데이터 저장. 이때 index=True를 하여 인덱스로 설정하였던 state와 city가 삭제되는 것을 방지
city_group_mean.to_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/(FT_작성)R_python_Code/state_citypop.csv', header=True, index=True,encoding='utf-8-sig')
# 데이터 로딩
city_group_mean=pd.read_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/(FT_작성)R_python_Code/state_citypop.csv')
city_group_mean
state_group=city_group_mean.set_index(['state']).groupby(level=[0]) # state를 기준으로 index
state_group_sum=state_group.sum() # 같은 state의 도시들을 기준으로 state_pop을 계산
state_group_sum.rename(columns={'city_pop':'state_pop'}, inplace=True) # 연산은 계속 city_pop으로 되왔기 때문에 state_pop으로 이름 변경
state_pop_merge=pd.merge(state_pop,state_group_sum, how='left',on='state') # 데이터 병합

In [ ]:
# 병합된 데이터 확인
state_pop_merge
# 데이터 저장
state_pop_merge.to_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/(FT_작성)R_python_Code/state_pop_merge.csv', header=True, index=False,encoding='utf-8-sig')

In [ ]:
# 생성된 변수 등 필요한 변수 정리
card_fraud_data=raw_card_data[['V1','category','gender','is_fraud']]
card_fraud_data

In [ ]:
# 저장했던 데이터 불러오기
differ_km=pd.read_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/(FT_작성)R_python_Code/distance_data.csv')
differ_time_past=pd.read_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/(FT_작성)R_python_Code/differ_time_past.csv')
differ_distance_past=pd.read_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/(FT_작성)R_python_Code/differ_distance_past.csv')
differ_amt=pd.read_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/(FT_작성)R_python_Code/differ_amt.csv')
state_pop=pd.read_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/(FT_작성)R_python_Code/state_pop_merge.csv')
print(differ_km.dtypes)
print(differ_time_past.dtypes)
print(differ_distance_past.dtypes)
print(differ_amt.dtypes)
print(state_pop.dtypes)

In [ ]:
# 변수 중복을 위한 필요한 데이터만 정리
differ_km=differ_km[['V1','lat','long','merch_lat','merch_long','differ_km']]
differ_distance_past=differ_distance_past[['V1','differ_past_km']]
differ_time_past=differ_time_past[['V1','trans_date_trans_time','differ_time']]
state_pop=state_pop[['V1','state','state_pop']]
# V1 key값을 기준으로 데이터 병합
analysis_card_data=pd.merge(card_fraud_data,differ_km, how='left',on='V1')
analysis_card_data=pd.merge(analysis_card_data,differ_distance_past, how='left',on='V1')
analysis_card_data=pd.merge(analysis_card_data,differ_time_past, how='left',on='V1')
analysis_card_data=pd.merge(analysis_card_data,differ_amt, how='left',on='V1')
analysis_card_data=pd.merge(analysis_card_data,state_pop, how='left',on='V1')
analysis_card_data

In [ ]:
# spop_fraud 변수를 통해 state별로 얼마나 부정 사용이 일어나는지의 비율을 계산
analysis_card_data['spop_fraud']=analysis_card_data['is_fraud']/analysis_card_data['state_pop']
# 최종 분석 데이터 저장
analysis_card_data.to_csv('/content/drive/MyDrive/google drive file/work/(책) 데이터 분석/통합/kaggle 데이터 분석 실습/(FT_작성)R_python_Code/analysis_card_data.csv', header=True, index=False,encoding='utf-8-sig')

In [ ]:
analysis_card_data

In [ ]:
# 데이터 타입 확인
print(analysis_card_data.dtypes)
print(analysis_card_data.shape)

In [ ]:
# is_fraud int로 되어있기 때문에 범주형 변수로 변경
analysis_card_data['is_fraud']=analysis_card_data['is_fraud'].astype('object')

In [ ]:
print(analysis_card_data.dtypes)

# 데이터 탐색



> 새로운 변수들을 만든 이후 분석에 필요한 데이터만을 이용하여 부정 탐지를 위한 데이터 분석을 실시.



In [ ]:
# 명목형 변수 빈도와 퍼센트
#fap=frequency and percent
# 이런 방식으로 각 범주형 열에 대한 빈도 분석
fap_cate=pd.DataFrame()
fap_cate['freq']=analysis_card_data['category'].value_counts()
fap_cate['per']=analysis_card_data['category'].value_counts()/len(analysis_card_data['category'])*100
fap_cate

In [ ]:
# 부정사용에 대한 여부 빈도 분석
fap_fraud=pd.DataFrame()
fap_fraud['freq']=analysis_card_data['is_fraud'].value_counts()
fap_fraud['per']=analysis_card_data['is_fraud'].value_counts()/len(analysis_card_data['is_fraud'])*100
fap_fraud

In [ ]:
# 연속형 변수에 대한 기술통계 정보 출력을 위해 describe() 함수를 사용
# 지수표현 없이 출력하기
pd.options.display.float_format='{:8f}'.format

analysis_card_data.describe()

In [ ]:
# 데이터 시각화를 위한 패키지 탑재
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = [10, 6]

In [ ]:
# state별로 평균적인 부정 사용 비율을 시각화하여 비교
# state는 중복적인 개념이기 때문에 이를 정리하기위한 작업을 실시.
state_fraud_mean=analysis_card_data.groupby('state').spop_fraud.mean()
pd.options.display.float_format = '{:.10f}'.format # 소수점 10의 자리까지 표시
state_fraud_mean

In [ ]:
# 시각화 x축을 위해 라벨링 이때 등장 순서로 되기 때문에 정렬 후, 유일값을 라벨링
state_label=analysis_card_data['state'].sort_values(ascending=True).unique()
state_label
import numpy as np
index=np.arange(len(state_label)) # x축 입력을 위해 초기 수치화

In [ ]:
# x축과 y축 설정
plt.bar(index, state_fraud_mean)
plt.title('state compare')
plt.xlabel('State', fontsize=15)
plt.ylabel('mean of fraud rate')
plt.xticks(index, label, fontsize=6)
plt.show()

In [ ]:
# 산점도
is_fraud_type=analysis_card_data['is_fraud'].unique()
plt.scatter(analysis_card_data['differ_past_km'],analysis_card_data['differ_amt'])
plt.xlabel('differ_past_km')
plt.ylabel('differ_amt')
plt.show()

# 종속변수와의 연관성 분석



> 범주형태인 종속벼수와의 연관성 분석을 위해 연속형 변수의 경우에는 t-teat분석을 실시하였으며 범주형 독립변수는 교차분석을 실시



In [ ]:
from scipy import stats #t-test 패키지 탑재
import numpy as np # 두 집단의 평균의 차이를 두하기 위한 패키지 탑재
fraud_no=analysis_card_data[(analysis_card_data['is_fraud']==0)]
fraud_yes=analysis_card_data[(analysis_card_data['is_fraud']==1)]

# differ_time에 대한 t-test
MeanDiffernce=np.mean(fraud_yes["differ_time"])-np.mean(fraud_no["differ_time"])
t_test_result=stats.ttest_ind(fraud_yes["differ_time"], fraud_no["differ_time"])

print("Mean difference between two groups: %.3f" % MeanDiffernce)
print("The t-statistic and p-value assuming equal variances is %.3f and %.3f." % t_test_result)

In [ ]:
# 거주지와 사용지 차이에 대한 차이 'differ_km'의 t-test
fraud_no=analysis_card_data[(analysis_card_data['is_fraud']==0)]
fraud_yes=analysis_card_data[(analysis_card_data['is_fraud']==1)]

# differ_time에 대한 t-test
MeanDiffernce=np.mean(fraud_yes["differ_km"])-np.mean(fraud_no["differ_km"])
t_test_result=stats.ttest_ind(fraud_yes["differ_km"], fraud_no["differ_km"])

print("Mean difference between two groups: %.3f" % MeanDiffernce)
print("The t-statistic and p-value assuming equal variances is %.3f and %.3f." % t_test_result)

In [ ]:
# 범주형 데이터  사이의 연관관계 파악을 위한 교차분석
!pip install researchpy
import researchpy as rp # 교차분석을 위한 패키지 설치

In [ ]:
# 'category'와 종속 변수인 'is_fraud' 사이의 교차분석 실시
results = rp.crosstab(analysis_card_data['category'], analysis_card_data['is_fraud'], prop= 'col', test= 'chi-square')
print(results)

# 이런의 교차분석으로 범주형 사이의 연관성을 파악할 수 있음 

In [ ]:
# 로지스틱 회귀분석
# 로지스틱 회귀분석용 데이터셋
logistic_data=analysis_card_data[['is_fraud','category','gender','state','state_pop','differ_km','differ_past_km','differ_time','amt','differ_amt']]
# 예측 모델 구축용 데이터셋
predict_data=analysis_card_data[['is_fraud','category','gender','state','state_pop','differ_km','differ_past_km','differ_time','amt','differ_amt']]
logistic_data

In [ ]:
# 로지스틱 종속변수에는 다시 수치형으로 입력해야하기 떄문에 int형으로 변경
logistic_data['is_fraud']=logistic_data['is_fraud'].astype('int')
# 독립변수들은 더미형으로 변경
logistic_dummy=pd.get_dummies(logistic_data)
# 독립변수만을 추출
x=logistic_dummy.drop(columns=['is_fraud']) # 종속변수인 'is_fraud'는 제외
# 독립변수의 이름을 추출(로지스틱 모델에 입력을 위해서)
x_col=x.columns[0:]
x_col

In [ ]:
# 로지스틱 분석을 위한 패키지 탑재
import statsmodels.api as sm

In [ ]:
logit_model=sm.Logit(logistic_dummy['is_fraud'], logistic_dummy[x_col])

result=logit_model.fit()
result.summary()

# 예측모델 생성 및 성능 파악

In [ ]:
# 훈련/검증용 데이터 분리
from sklearn.model_selection import train_test_split # 데이터 분할 필요한 패키지
predict_data_=pd.get_dummies(predict_data)
# 정답 데이터 분리
X=predict_data.drop(['is_fraud'], axis=1)
y=predict_data[['is_fraud']]

In [ ]:
X=pd.get_dummies(X)

In [ ]:
# 데이터 스케일링을 통해 데이터 범위 정규화함으로써 더 좋은 성능을 기대
from sklearn.preprocessing import MinMaxScaler

#훈련용 데이터를 표준화된 데이터로 만듦으로써 데이터의 값이 정규분포 안에 속하도록 만든다.
features = X.columns.values
scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(X)
X = pd.DataFrame(scaler.transform(X))
X.columns = features
#훈련용 데이터를 표준화된 데이터로 만듦으로써 데이터의 값이 정규분포 안에 속하도록 만든다.
features = y.columns.values
scaler = MinMaxScaler(feature_range = (0,1))
scaler.fit(y)
y = pd.DataFrame(scaler.transform(y))
y.columns = features

In [ ]:
# 7:3 분할
X_train_7, X_test_7, y_train_7, y_test_7 = train_test_split(X, y, test_size= 0.3, random_state=1234)
print(X_train_7.shape)
# 5:5 분할
X_train_5, X_test_5, y_train_5, y_test_5 = train_test_split(X, y, test_size= 0.5, random_state=1234)
print(X_train_5.shape)

In [ ]:
X_train_7

In [ ]:
# 로지스틱 회귀기반의 머신러닝 예측 모델을 위한 패키지 탑재
from sklearn.linear_model import LogisticRegression

# 변수 중요도 추출을 위한 패키지 탑재
from sklearn import metrics

# 예측 모델 만들기
model = LogisticRegression()
result = model.fit(X_train_5, y_train_5)

#변수의 중요도를 도출하기 위한 패키지 설치와 중요도 도출하기
from sklearn import metrics

prediction_test = model.predict(X_test_5)
print (metrics.accuracy_score(y_test_5, prediction_test))
print (metrics.recall_score(y_test_5, prediction_test))
from sklearn.metrics import confusion_matrix
predict_result=confusion_matrix(y_test_5,prediction_test)
print(predict_result)
#sensitivity = predict_result[0,0]/(predict_result[0,0]+predict_result[0,1])
#specificity = predict_result[1,1]/(predict_result[1,0]+predict_result[1,1])
#print(sensitivity)
#print(specificity)
weights = pd.Series(model.coef_[0], index=X.columns.values)
print (weights.sort_values(ascending = False)[:].plot(kind='bar'))

In [ ]:
# 랜덤 포레스트 패키지 탑재 
from sklearn.ensemble import RandomForestClassifier
'''
# 훈련용 모델을 만드는 것으로 아직 데이터가 들어가지 않은 상태이다.
n_estimators=1000 생성할 트리의 수, 
n_jobs=-1이면 컴퓨터의 모든 코어를 사용하여 연산
max_features= auto는 매개변수(노드)는 자동으로 형성해라, 
max_leaf_nodes는 복잡도를 낮추는 함수
"e"f"e"f
'''
model_rf = RandomForestClassifier(n_estimators=1000 , n_jobs = -1,
random_state =50, max_features = "auto", max_leaf_nodes = 30)
#생성된 모델에 데이터를 탑재
model_rf.fit(X_train_5, y_train_5)
# 훈련된 랜덤 포레트스 예측 모델에 대한 검증을 실시
prediction_test = model_rf.predict(X_test_5)
# 정확도 출력
from sklearn import metrics
print (metrics.accuracy_score(y_test_5, prediction_test))
print (metrics.recall_score(y_test_5, prediction_test))
from sklearn.metrics import confusion_matrix
predict_result=confusion_matrix(y_test_5,prediction_test)
print(predict_result)
# 변수의 중요도를 출력
importances = model_rf.feature_importances_
weights = pd.Series(importances,index=X.columns.values)
weights.sort_values()[:].plot(kind = 'barh')

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import Dense

# 인공신경망 구조 만들기 Sequential은 케라스가 제공하는 모델링 구조 중 연속적 연결 구조에 해당
card_fraud_ANN = Sequential()
'''
kernel_initializer= 가중치 초기화 방법: 'uniform'(균일분포)/"enormal"f(가우시안 분포)
activation= 활성화 함수: 
'linear'(선형_가중치가 결과값 그대로)/'rele'(rectifier 함수_히든 레이어에 자주 사용)
"esigmoid"f(시그모이드 함수_이진 분류 문제에서 출력층)/"esoftmax"f(소프트맥스 함수_다중 분류 문제)
'''
# 독립변수로 부터 받는 첫 번째 히든 레이어 설정 50개의 노드를 생성, input_dim=독립변수의 갯 수, 4개 히든 레이어
card_fraud_ANN.add(Dense(units = 20, kernel_initializer = 'uniform', activation = 'relu', input_dim = 73))
card_fraud_ANN.add(Dense(units = 20, kernel_initializer = 'uniform', activation = 'relu'))
card_fraud_ANN.add(Dense(units = 20, kernel_initializer = 'uniform', activation = 'relu'))
card_fraud_ANN.add(Dense(units = 20, kernel_initializer = 'uniform', activation = 'relu'))
# 출력층 출력, 이진 분류에 대한 활성화 함수이기 때문에 시그모이드 함수를 사용
card_fraud_ANN.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [ ]:
# 예측 모델 학습과정 설정
'''
optimizer=손실을 줄이며 모델의 가중치 최적화 방법 설정, Gradient descent방식이 대표적이나, 최근 딥러닝에서는 adam이 자주 사용
loss= 손실을 계산하는 방법 오차 제곱의 평균(MSE:Mean Squared Error), 확률적 계산 방식인 Cross Entropy방법이 있다.
Cross Entropy=binary_crossentropy(종속변수가 2개의 범주)/categorical_crossentropy(종속변수 3개 이상의 범주)
'''
card_fraud_ANN.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) # 다른 방법 없음?
# 예측 모델 학습 설정 
'''
batch_size= 가중치 갱신할 데이터의 수 4900여개의 데이터를 분석하는데에 있어서 100개 분석한 후 100개의 데이터를 비교한 후 가중치 갱신
epochs = 같은 데이터를 몇 번 반복하여 학습할지 설정, epochs의 수가 많아서 overfitting문제 발생 시 학습 종류
'''
card_fraud_ANN.fit(X_train_5, y_train_5, batch_size = 100, epochs = 20)

In [ ]:
# 신경망 정확도 도출
results = card_fraud_ANN.evaluate(X_test_5, y_test_5)
print(results)
#print('Test accuracy: ', results[1])

In [ ]:
deep_y_predict=card_fraud_ANN.predict(X_test_5)
deep_y_predict
#deep_y_predict=(deep_y_predict>0.5,1,0)
#list(deep_y_predict)
#deep_y_predict

In [ ]:
deep_y_predict_05=deep_y_predict
deep_y_predict_05=(deep_y_predict_05>0.5)
deep_y=pd.DataFrame(deep_y_predict_05)
deep_y.rename(columns={0:'result'}, inplace=True)
deep_y

In [ ]:
predict_result=confusion_matrix(y_test_5,deep_y)
print(predict_result)
sensitivity = predict_result[0,0]/(predict_result[0,0]+predict_result[0,1])
specificity = predict_result[1,1]/(predict_result[1,0]+predict_result[1,1])
print(sensitivity)
print(specificity)